In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from six import StringIO  
from IPython.display import Image
import matplotlib.pyplot as plt
import random
import requests
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from scipy.spatial.distance import cosine
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers

import sys
import os
import json
import pandas
import numpy
import optparse
import keras


from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict
from PIL import Image

from scipy.spatial.distance import cosine

random.seed(1242)

C:\Users\tab\Documents\Code\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Testing for Overfitting

To test for overfitting, we will run the algorithm through another user profile and analyze the results. We will use the profile SimplySweet (user 26149) for this since its favorited animes differ from Zeroskye's profile, allowing us to see how the model performs on varied profile compositions.

The favorited animes in this profile are:
- 6045: Kimi ni Todoke
- 164: Mononoke Hime
- 431: Howl no Ugoku Shiro
- 202: Wolf's Rain

In [2]:
user_anime_dataset = pd.read_csv('user_anime_dataset.csv')

In [3]:
# Create a function to calculate cosine similarity between two items
def calculate_cosine_similarity(item1, item2):
    return 1 - cosine(item1, item2)

In [4]:
from scipy.spatial.distance import cosine

# Select user 26149 (SimplySweet)
target_user_id = 26149
user_7968 = user_anime_dataset[user_anime_dataset['user_id'] == target_user_id].set_index('user_id')
user_7968_animes = user_7968.columns[user_7968.values.flatten() == 1]

# Calculate cosine similarity with all users
similarities = pd.DataFrame(index=user_anime_dataset['user_id'].unique(), columns=['Similarity'])

for index, row in user_anime_dataset.iterrows():
    similarity = calculate_cosine_similarity(user_7968.values.flatten(), row.values[1:])
    similarities.loc[row['user_id']] = similarity

# Find the top 5 most similar users
top_5_most_similar = similarities.sort_values(by='Similarity', ascending = False).head(5)

# Initialize an empty DataFrame for recommendation scores
recommendation_scores = pd.DataFrame(index = user_anime_dataset['user_id'].unique(), columns = user_anime_dataset.columns[1:])

# Fill in recommendation scores for each user and anime
for index, row in user_anime_dataset.iterrows():
    if row['user_id'] == target_user_id:
        continue  # Skip the target user
    
    # Calculate recommendation score using a combination of user and item approach
    recommendation_scores.loc[row['user_id']] = row.values[1:] * similarities.loc[row['user_id'], 'Similarity']

# Sum recommendation scores across similar users for each anime
anime_recommendation_scores = recommendation_scores.sum(axis = 0)

# Exclude animes that the target user has already seen
anime_recommendation_scores = anime_recommendation_scores[anime_recommendation_scores.index.isin(user_7968_animes) == False]

# Sort and get the top 10 recommended animes
top_10_recommendations = anime_recommendation_scores.sort_values(ascending = False).head(10)

# Convert the variable to a string
top_10_recommendations_str = ', '.join(map(str, top_10_recommendations.index))

# Display the top 5 most similar users and top 10 recommended animes
print(f"The top 5 most similar users to user {target_user_id} are {top_5_most_similar}.")
print(f"We would recommend the following based on a combination of user-item approach: {top_10_recommendations_str}. \n")


The top 5 most similar users to user 26149 are       Similarity
26149          1
48735          1
31759          1
61449    0.67082
76012   0.612372.
We would recommend the following based on a combination of user-item approach: anime_5114, anime_1, anime_199, anime_1535, anime_9253, anime_30, anime_11061, anime_4181, anime_457, anime_1575. 



The recommended animes are:

- 5114: Fullmetal Alchemist: Brotherhood
- 1: Cowboy Bebop
- 199: Sen to Chihiro no Kamikakushi
- 1535: Death Note
- 9253: Steins;Gate
- 30: Offside
- 11061: Hunter x Hunter (2011)
- 4181: Clannad: After Story
- 457: Mushishi
- 1575: Code Geass: Hangyaku no Lelouch
